In [1]:
import os
import pydot
import graphviz
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-03-14 16:26:29.004524: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 16:26:30.038709: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib.real:/opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/python3.8/site-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-14 16:26:30.038799: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.

In [3]:
image_size = (128, 128)
batch_size = 16


train_ds = tf.keras.utils.image_dataset_from_directory(
    "/workspace/Shantan_Data/Bayesian2v/Auto1/Dataset1_1",
    validation_split=None,
    subset=None,
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "/workspace/Shantan_Data/Bayesian2v/Auto1/Dataset2_1",
    validation_split=None,
    subset=None,
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
print(train_ds)
print(val_ds)


Found 30 files belonging to 2 classes.
Found 30 files belonging to 2 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [6]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
keras.utils.plot_model(model, show_shapes=True)

epochs = 20

#callbacks = [keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),]
model.compile(
    optimizer=keras.optimizers.Adam(0.01),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Epoch 1/20
2/2 [==============================] - 7s 489ms/step - loss: 1.7007 - accuracy: 0.3000 - val_loss: 1.0331 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 0s 149ms/step - loss: 1.2804 - accuracy: 0.5000 - val_loss: 0.6475 - val_accuracy: 0.5000
Epoch 3/20
2/2 [==============================] - 0s 152ms/step - loss: 1.4304 - accuracy: 0.5000 - val_loss: 2.1280 - val_accuracy: 0.5000
Epoch 4/20
2/2 [==============================] - 0s 161ms/step - loss: 0.7507 - accuracy: 0.5667 - val_loss: 0.6183 - val_accuracy: 0.5000
Epoch 5/20
2/2 [==============================] - 0s 153ms/step - loss: 1.1215 - accuracy: 0.5000 - val_loss: 0.7429 - val_accuracy: 0.5000
Epoch 6/20
2/2 [==============================] - 0s 153ms/step - loss: 0.8538 - accuracy: 0.5000 - val_loss: 4.3101 - val_accuracy: 0.5000
Epoch 7/20
